# COMP4160_Yinjie_Liu_20211091_Assignment1_2

In this assignment we will collect detailed specific energy imports and exports data from the **US Energy Information Administration API** for five different energy resources.

This notebook covers Task 2 - **Data Preparation and Analysis**. We will use all of the JSON data collected during Task 1 to perform the analysis process.

In [ ]:
import numpy as np
import json
from pathlib import Path
from datetime import datetime
import pandas as pd
from sklearn.metrics.pairwise import euclidean_distances
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import matplotlib.ticker as ticker
from pandas.plotting import scatter_matrix
import seaborn as sns
%matplotlib inline

Key settings, including our list of resources and various appearance settings for plotting later:

In [ ]:
# The city locations that we would like to study
imports_names = ["TOTAL.BFIMBUS.A", "TOTAL.CCIMBUS.A", "TOTAL.CLIMBUS.A", "TOTAL.ELIMBUS.A", "TOTAL.COIMBUS.A"]

exports_names = ["TOTAL.BMEXBUS.A", "TOTAL.CCEXBUS.A", "TOTAL.CLEXBUS.A", "TOTAL.ELEXBUS.A", "TOTAL.COEXBUS.A"]
# Precision that we'll use for Pandas display
pd.set_option('precision', 2)
# Colors that we will use in various plots
plots_colors = {"TOTAL.BFIMBUS.A":"green", "TOTAL.CCIMBUS.A":"darkred", "TOTAL.CLIMBUS.A":"orange", "TOTAL.ELIMBUS.A":"navy", "TOTAL.COIMBUS.A":"red"}
plots_colors2 = {"TOTAL.BMEXBUS.A":"green", "TOTAL.CCEXBUS.A":"darkred", "TOTAL.CLEXBUS.A":"orange", "TOTAL.ELEXBUS.A":"navy", "TOTAL.COEXBUS.A":"red"}
# Default font size
fontsize = 13

Directories for data storage:

In [ ]:
# directory for raw data storage
dir_raw = Path("raw")
# directory for storing clean pre-processed data
dir_data = Path("data")
# make sure it exists
dir_data.mkdir(parents=True, exist_ok=True)

# Data Preprocessing

Before we can analyse the data, we need to transform it into a usable format.

Firstly, find all of the raw JSON files and separate them by resources:

In [ ]:
resource_files = {}
resource_files["Imports"] = []
resource_files["Exports"] = []
for fpath in dir_raw.iterdir():
    if fpath.suffix == ".json":
        # parse the filename
        # parts = fpath.stem.split("-")
        if "Imports" in str(fpath):
            resource_files["Imports"].append(fpath)
        elif "Exports" in str(fpath):
            resource_files["Exports"].append(fpath)

print("Imports: Found %d raw data files" % (len(resource_files["Imports"])))
print("Exports: Found %d raw data files" % (len(resource_files["Exports"])))

## Data Parsing and Merging

We will define a function that will parse an individual raw JSON data file coming from the **US Energy Information Administration API** .

An important aspect with this API is how we handle the relationship between index and column. The original data's index are four different resources, however, sometimes we need the column(year) to be index since the visualized graphs will be more readable by this way.
Another important problem is that there are missing values in the dataset, we need to find a proper way to fill in there data.

In [ ]:
def parse_raw_data(Theme, fpath):
    fin = open(fpath, "r")
    jdata = fin.read()
    data = json.loads(jdata)
    # handle each observation
    parts = fpath.stem.split("-")
    rows = []
    
    row = {"Title": parts[0]}
    row["Theme"] = Theme
    # add temperature data, in celsius - both raw and 'feels like' with wind chill
    row["Frequency"] = data["series"][0]["f"]
    row["units"] = data["series"][0]["units"]
    # add precpipation level, in mm for last hour
    row["start"] = data["series"][0]["start"]
    # add wind speed
    row["end"] = data["series"][0]["end"]
    index = 0
    len_list = len(data["series"][0]["data"])- 1
    # only pick up six years' data for demonstrated purpose
    for i in range(len_list,-1,-1):
        row[data["series"][0]["data"][i][0]] = data["series"][0]["data"][i][1]
    rows.append(row)
    fin.close()
    # return back the DataFrame
    return pd.DataFrame(rows)

Define a function that will parse and merge a collection of raw data files:

In [ ]:
def parse_merge_raw_data(Theme, fpaths):
    df_files = []
    # process each file for Imports and Exports theme
    print("Reading and parsing %d JSON data files ..." % len(fpaths))
    for fpath in fpaths:
        # produce a DataFrame from this file
        df_file = parse_raw_data(Theme, fpath)
        df_files.append(df_file)
    # concatenate all the DataFrames together into a single frame
    df_resources = pd.concat(df_files)
    # now set the index column to be the Date, which is now unique
    df_resources.set_index("Title", inplace=True)
    # we will also sort the frame by its index
    return df_resources.sort_index()    

Parse and merge the raw data files for each theme, producing a single merged Pandas Data Frames for each theme which we can analyse later on:

In [ ]:
df_resources = {}
print("-- Imports")
df_resource = parse_merge_raw_data("Imports", resource_files["Imports"])
df_resources["Imports"] = df_resource

print("-- Exports")
df_resource = parse_merge_raw_data("Exports", resource_files["Exports"])
df_resources["Exports"] = df_resource

In [ ]:
df_resources["Imports"].head(5)

Check the number of rows and resourece names in the DataFrames:

In [ ]:
Theme_names = ["Imports", "Exports"]

# This function will show which resources' information we have right now.
def summarize_dates():
    rows = []
    for Theme_name in Theme_names:
        row = {"Theme": Theme_name}
        row["Rows"] = len(df_resources[Theme_name])
        count = 1
        for resoure_name in list(df_resources[Theme_name].index):
            key_name = "Example" + str(count)
            row[key_name] = resoure_name
            count+=1
        rows.append(row)
    return pd.DataFrame(rows).set_index("Theme")

# display the ranges
df_date_summary = summarize_dates()
df_date_summary

Find the latest start year and the most earlier year

In [ ]:
# find the latest start date
date_start = df_resources["Imports"]["start"].max()
# find the earlier end date
date_end = df_resources["Exports"]["end"].max()
date_start, date_end

Find out how many missing values there are bwtween two themes

In [ ]:
for Theme_name in Theme_names:
    print("-- %s" % Theme_name)
    # replace the NA vlaue with np.nan
    df_resources[Theme_name].replace("NA", np.nan, inplace = True)
    missing_counts = df_resources[Theme_name].isnull().sum()
    # which coludmns have missing values?
    missing_counts = missing_counts[missing_counts>0]
    
    if len(missing_counts) == 0:
        print("No missing values found")
    else:
        print("There are total %d missing values found in the data frame" % len(missing_counts))
        

In [ ]:
# fill in the data with 0 which is np.nan
for Theme_name in Theme_names:
    df_resources[Theme_name] = df_resources[Theme_name].fillna(0)

In [ ]:
df_resources["Exports"]

In [ ]:
df_resources["Imports"]

Reversing the datagrames so that years can be index.

In [ ]:
# delete some useless column and reverse this dataframe and make years be index.
imports_dataframe = df_resources["Imports"].drop(["Theme", "Frequency","units", "start", "end"], axis=1)
data_list = imports_dataframe.values.tolist()

rows = []
count = 0
# the latest year and most earlier year
for i in range(1949, 2021):
    row = {"Year": i}

    row["TOTAL.BFIMBUS.A"] = data_list[0][count]
    row["TOTAL.CCIMBUS.A"] = data_list[1][count]
    row["TOTAL.CLIMBUS.A"] = data_list[2][count]
    row["TOTAL.COIMBUS.A"] = data_list[3][count]
    row["TOTAL.ELIMBUS.A"] = data_list[4][count]
    row["Theme"] = "Imports"
    row["units"] = "Trillion Btu"
    rows.append(row)
    count+=1
new_imports_dataframe = pd.DataFrame(rows)
# set the year as index
new_imports_dataframe.set_index("Year", inplace=True)

df_resources["Imports"] = new_imports_dataframe.sort_index()    

In [ ]:
# delete some useless column and reverse this dataframe and make years be index.
exports_dataframe = df_resources["Exports"].drop(["Theme", "Frequency","units", "start", "end"], axis=1)
data_list = exports_dataframe.values.tolist()

rows = []
count = 0
# the latest year and most earlier year
for i in range(1949, 2021):
    row = {"Year": i}
    
    row["TOTAL.BMEXBUS.A"] = data_list[0][count]
    row["TOTAL.CCEXBUS.A"] = data_list[1][count]
    row["TOTAL.CLEXBUS.A"] = data_list[2][count]
    row["TOTAL.COEXBUS.A"] = data_list[3][count]
    row["TOTAL.ELEXBUS.A"] = data_list[4][count]
    row["Theme"] = "Exports"
    row["units"] = "Trillion Btu"
    rows.append(row)
    count+=1
new_exports_dataframe = pd.DataFrame(rows)
# set the year as index
new_exports_dataframe.set_index("Year", inplace=True)

df_resources["Exports"] = new_exports_dataframe.sort_index()    

# Data Saving

Now write out all of the merged themes DataFrames in JSON format, one file per theme. These will represent the 'clean' data that we will use next for our actual data analysis:

In [ ]:

for Theme_name in Theme_names:
    fname = "%s.json" % Theme_name
    out_path = dir_data / fname
    print("Writing %s" % out_path)
    df_resources[Theme_name].to_json(out_path, orient="index", indent=4)

In [ ]:
df_resources["Imports"]

In [ ]:
df_resources["Exports"]

# Data Analysis

## Data Loading

We will reload the resources data frames that we stored previously.

In [ ]:
# it will show how many rows we have between two themes and the index year information
df_resources = {}
for Theme_name in Theme_names:
    fname = "%s.json" % Theme_name
    in_path = dir_data / fname
    df_resources[Theme_name] = pd.read_json(in_path, orient="index")
    print("%s: %d rows of data - [%s to %s]" % 
          (Theme_name, len(df_resources[Theme_name]), 
           df_resources[Theme_name].index.min(), df_resources[Theme_name].index.max() ))

In [ ]:
df_resources["Imports"].head(5)

## Data Analysis

we look at resources data. We could just look at the raw statistics for the different themes to calculate their maximum, minimum and mea. 

In [ ]:
def imports_compute_stats():
    row = {}
    imports_rows = []
    for imports_name in imports_names:
        row = {"Title": imports_name}
        row["Min"] = df_resources["Imports"][imports_name].min()
        row["Max"] = df_resources["Imports"][imports_name].max()
        row["Mean"] = df_resources["Imports"][imports_name].mean()
        row["Median"] = df_resources["Imports"][imports_name].median()
        imports_rows.append(row)
    return pd.DataFrame(imports_rows).set_index("Title")

In [ ]:
def exports_compute_stats():
    row = {}
    exports_rows = []
    for exports_name in exports_names:
        row = {"Title": exports_name}
        row["Min"] = df_resources["Exports"][exports_name].min()
        row["Max"] = df_resources["Exports"][exports_name].max()
        row["Mean"] = df_resources["Exports"][exports_name].mean()
        row["Median"] = df_resources["Exports"][exports_name].median()
        exports_rows.append(row)
    return pd.DataFrame(exports_rows).set_index("Title")

In [ ]:
imports_compute_stats()

In [ ]:
exports_compute_stats()

We could look at plots for each of the different themes separately based on different resources:

In [ ]:
fig = plt.figure(figsize=(12,7))
for i, resource_name in enumerate(imports_names):
    if resource_name == "TOTAL.COIMBUS.A":
        pass
    else:
        axc = fig.add_subplot(2, 2, i+1)
        ax = df_resources["Imports"][resource_name].plot(ax=axc, legend=False, lw=2, color=plots_colors[resource_name])
        ax.set_title(resource_name)
        ax.set_ylabel("Trillion Btu")
        # note that we will use consistent Y-axis limits based on the ranges we computed above
        ax.set_ylim(df_resources["Imports"][resource_name].min()-10 , df_resources["Imports"][resource_name].max() + 10)
        ax.set_xlim(df_resources["Imports"].index.min(), df_resources["Imports"].index.max())
# fix the spacing between plots
fig.tight_layout()

Separate the TOTAL.COIMBUS.A data since its data is too bigger than other resources which will lead to illustration issues.

In [ ]:
ax = df_resources["Imports"]["TOTAL.COIMBUS.A"].plot(legend=False, lw=2, color=plots_colors["TOTAL.COIMBUS.A"])
ax.set_title("TOTAL.COIMBUS.A")
ax.set_ylabel("Trillion Btu")
# note that we will use consistent Y-axis limits based on the ranges we computed above
ax.set_ylim(df_resources["Imports"]["TOTAL.COIMBUS.A"].min()-1000 , df_resources["Imports"]["TOTAL.COIMBUS.A"].max() + 1000)
ax.set_xlim(df_resources["Imports"].index.min(), df_resources["Imports"].index.max())

From the plots above, we can see the imported resources trends. In summary, only ELIMBUS is stable and it had been increasing from 1949 till 2020. As for the other resources, they all have experienced up and down to a great extent.

In [ ]:
plots_colors2 = {"TOTAL.BMEXBUS.A":"green", "TOTAL.CCEXBUS.A":"darkred", "TOTAL.CLEXBUS.A":"orange", "TOTAL.ELEXBUS.A":"navy", "TOTAL.COEXBUS.A":"red"}

Below code is for export graphs, separate the TOTAL.COEXBUS.A data since its data is too bigger than other resources which will lead to illustration issues.

In [ ]:
fig = plt.figure(figsize=(12,7))
for i, resource_name in enumerate(exports_names):
    if resource_name == "TOTAL.COEXBUS.A":
        pass
    else:
        axc = fig.add_subplot(2, 2, i+1)
        ax = df_resources["Exports"][resource_name].plot(ax=axc, legend=False, lw=2, color=plots_colors2[resource_name])
        ax.set_title(resource_name)
        ax.set_ylabel("Trillion Btu")
        # note that we will use consistent Y-axis limits based on the ranges we computed above
        ax.set_ylim(df_resources["Exports"][resource_name].min()-10 , df_resources["Exports"][resource_name].max() + 100)
        ax.set_xlim(df_resources["Exports"].index.min(), df_resources["Exports"].index.max())
# fix the spacing between plots
fig.tight_layout()

In [ ]:
ax = df_resources["Exports"]["TOTAL.COEXBUS.A"].plot(legend=False, lw=2, color=plots_colors2["TOTAL.COEXBUS.A"])
ax.set_title("TOTAL.COEXBUS.A")
ax.set_ylabel("Trillion Btu")
# note that we will use consistent Y-axis limits based on the ranges we computed above
ax.set_ylim(df_resources["Exports"]["TOTAL.COEXBUS.A"].min()-1000 , df_resources["Exports"]["TOTAL.COEXBUS.A"].max() + 1000)
ax.set_xlim(df_resources["Exports"].index.min(), df_resources["Exports"].index.max())

From the plots above, we can see the exported resources trends. In summary, only BMEXBUS is qurite stable and it had kept around 0. As for the other resources, CLEXBUS have experienced up and down to a great extent. And COEXBUS had dramatically incresed from 0 to more than 6000.

There are several bar graphs to show resoures tendency from 1949 to 2020 amongest two themes.

In [ ]:
fig = plt.figure(figsize=(20,25))
for i, resource_name in enumerate(exports_names):
    axc = fig.add_subplot(5, 1, i+1)
    ax = df_resources["Exports"][resource_name].plot(ax=axc, kind="bar", legend=False, lw=2, color=plots_colors2[resource_name])
    ax.set_title(resource_name)
    ax.set_ylabel("Trillion Btu")
    ax.yaxis.grid(True);
fig

What we can see from above bar grahps is that CCEXBUS and CLEXBUS are always most popular resources to export from American. And BMEXBUS is getting popular recently.

In [ ]:
fig = plt.figure(figsize=(20,25))
for i, resource_name in enumerate(imports_names):
    axc = fig.add_subplot(5, 1, i+1)
    ax = df_resources["Imports"][resource_name].plot(ax=axc, kind="bar", legend=False, lw=2, color=plots_colors[resource_name])
    ax.set_title(resource_name)
    ax.set_ylabel("Trillion Btu")
    ax.yaxis.grid(True);
    
# fix the spacing between plots
fig.tight_layout()

What we can see from above bar grahps is that ELIMBUS and COIMBUS are always most popular resources to import to American. And BFIMBUS is getting popular recently.

At the same time, there is another way to draw linear graphs to show the tendency about different resources.

This method doesn't have to separate the TOTAL.COIMBUS.A data from others.

In [ ]:
ax = df_resources["Imports"].plot(kind='line', subplots=True, layout=(3,3), figsize=(15,10))

In [ ]:
ax = df_resources["Exports"].plot(kind='line', subplots=True, layout=(3,3), figsize=(15,10))

And then we pick up the minimal and maximal data from different resources and combine them into a new Dataframe.

But we separate the COIMBUS's data as a result of same result as above linear graphs.

In [ ]:
df_resource_data = {}
min_temps = []
max_temps = []
min_COIMBUS= []
max_COIMBUS = []
for imports_name in imports_names:
    
    # add min and max data into list group by resource names
    min_temps.append(df_resources["Imports"][imports_name].min())
    max_temps.append(df_resources["Imports"][imports_name].max())
    # combine the min and max values into a new DataFrame
    
min_COIMBUS.append(min_temps.pop())
max_COIMBUS.append(max_temps.pop())
df_resource_data["Imports"] = pd.DataFrame({"Min": min_temps, "Max": max_temps})
df_resource_data["Imports"].index = imports_names[0:4]
df_COIMBUS_data = pd.DataFrame({"Min": min_COIMBUS, "Max": max_COIMBUS})
df_COIMBUS_data.index = imports_names[4:5]

In [ ]:
df_resource_data["Imports"]

In [ ]:
df_COIMBUS_data

Then we can plot these individually, with comparative bar plots for each resources showing the minimum and maximum data:

In [ ]:
fig = plt.figure(figsize=(12,8))
axc = fig.add_subplot(1,2,1)
ax = df_resource_data["Imports"].plot.bar(ax=axc, y=['Min', 'Max'], 
                                        color={ "Min": "royalblue", "Max": "firebrick"}, fontsize=fontsize, zorder=3)
ax.set_title("The Max and Mini data based on resoureces:")
ax.set_ylabel("Trillion Btu")
# show the exact number on the each bar
for a,b in zip(np.arange(4),df_resource_data["Imports"]["Min"]):
    ax.text(a-0.13,b+0.2,b,ha='center',va='bottom',fontsize=10)

for a,b in zip(np.arange(4),df_resource_data["Imports"]["Max"]):
    ax.text(a+0.13,b+0.2,b,ha='center',va='bottom',fontsize=10)

# Configure grid lines
ax.yaxis.grid(True);

axc = fig.add_subplot(1,2,2)
ax = df_COIMBUS_data.plot.bar(ax=axc, y=['Min', 'Max'], 
                                        color={ "Min": "royalblue", "Max": "firebrick"}, fontsize=fontsize, zorder=3)

ax.set_title("The COIMBUS's Max and Mini data")
ax.set_ylabel("Trillion Btu")

# show the exact number on the each bar
for a,b in zip(np.arange(1),df_COIMBUS_data["Max"]):
    ax.text(a+0.13,b+0.2,b,ha='center',va='bottom',fontsize=10)
for a,b in zip(np.arange(1),df_COIMBUS_data["Min"]):
    ax.text(a-0.13,b+0.2,b,ha='center',va='bottom',fontsize=10)

# Configure grid lines
ax.yaxis.grid(True);

fig.tight_layout()

And then we pick up the minimal and maximal data from different resources and combine them into a new Dataframe.

But we separate the COEXBUS's data and CLEXBUS's data as a result of same result as above linear graphs.

In [ ]:
df_resource_data = {}
min_temps = []
max_temps = []
min_COEXBUS_CLEXBUS= []
max_COEXBUS_CLEXBUS = []
for exports_name in exports_names:
    
    # add min and max data into list group by resource names
    min_temps.append(df_resources["Exports"][exports_name].min())
    max_temps.append(df_resources["Exports"][exports_name].max())
    # combine the min and max values into a new DataFrame
    
min_COEXBUS_CLEXBUS.append(min_temps.pop(2))
min_COEXBUS_CLEXBUS.append(min_temps.pop(3))

max_COEXBUS_CLEXBUS.append(max_temps.pop(2))
max_COEXBUS_CLEXBUS.append(max_temps.pop(3))
df_resource_data["Exports"] = pd.DataFrame({"Min": min_temps[0:3], "Max": max_temps[0:3]})
df_resource_data["Exports"].index = exports_names[0:3]
df_COEXBUS_CLEXBUS_temps = pd.DataFrame({"Min": min_COEXBUS_CLEXBUS, "Max": max_COEXBUS_CLEXBUS})
df_COEXBUS_CLEXBUS_temps.index = exports_names[3:5]

In [ ]:
df_resource_data["Exports"]

In [ ]:
df_COEXBUS_CLEXBUS_temps

In [ ]:
fig = plt.figure(figsize=(12,8))
axc = fig.add_subplot(1,2,1)
ax = df_resource_data["Exports"].plot.bar(ax=axc, y=['Min', 'Max'], 
                                        color={ "Min": "royalblue", "Max": "firebrick"}, fontsize=fontsize, zorder=3)
ax.set_title("The Max and Mini data based on resoureces:")
ax.set_ylabel("Trillion Btu")
# show the exact number on the each bar
for a,b in zip(np.arange(4),df_resource_data["Exports"]["Min"]):
    ax.text(a-0.13,b+0.2,b,ha='center',va='bottom',fontsize=10)

for a,b in zip(np.arange(4),df_resource_data["Exports"]["Max"]):
    ax.text(a+0.13,b+0.2,b,ha='center',va='bottom',fontsize=10)

# Configure grid lines
ax.yaxis.grid(True);

axc = fig.add_subplot(1,2,2)
ax = df_COEXBUS_CLEXBUS_temps.plot.bar(ax=axc, y=['Min', 'Max'], 
                                        color={ "Min": "royalblue", "Max": "firebrick"}, fontsize=fontsize, zorder=3)

ax.set_title("The COIMBUS's Max and Mini data")
ax.set_ylabel("Trillion Btu")

# show the exact number on the each bar
for a,b in zip(np.arange(2),df_COEXBUS_CLEXBUS_temps["Max"]):
    ax.text(a+0.13,b+0.2,b,ha='center',va='bottom',fontsize=10)
for a,b in zip(np.arange(2),df_COEXBUS_CLEXBUS_temps["Min"]):
    ax.text(a-0.13,b+0.2,b,ha='center',va='bottom',fontsize=10)

# Configure grid lines
ax.yaxis.grid(True);

fig.tight_layout()

As we can see from above graphs that COEXBUS had most increase during 70 years and BMEXBUS almost didn't changed at all.

Then we can draw the lines in a single graph about tendency of import based on resources. 

In [ ]:
ax = df_resources["Imports"].drop(["TOTAL.COIMBUS.A"], axis=1).plot(figsize=(14,5.5), color=plots_colors, fontsize=fontsize, zorder=3)
ax.set_title("Import data information about four resources", fontsize=fontsize)
ax.set_xlabel("Year", fontsize=fontsize)
ax.set_ylabel("Trillion Btu", fontsize=fontsize)
# adjust the axis limits
ax.set_xlim(df_resources["Imports"].index.min(), df_resources["Imports"].index.max())
# move the legend outside the plot
ax.legend(bbox_to_anchor=(1.18, 1), fontsize=fontsize)
# add an x grid
ax.xaxis.grid(True);

In [ ]:
ax = df_resources["Imports"]["TOTAL.COIMBUS.A"].plot(figsize=(14,5.5), color=plots_colors, fontsize=fontsize, zorder=3)
ax.set_title("Temperature Over Time", fontsize=fontsize)
ax.set_xlabel("Date", fontsize=fontsize)
ax.set_ylabel("Temperature (C)", fontsize=fontsize)
# adjust the axis limits
# ax.set_ylim(0 , df_cities["Imports"]["TOTAL.COIMBUS.A"].max() + 10)
ax.set_xlim(df_resources["Imports"].index.min(), df_resources["Imports"].index.max())
# change the format of the dates on the x-axis
# ax.xaxis.set_major_formatter(mdates.DateFormatter('%d/%m'))    
# move the legend outside the plot
ax.legend(bbox_to_anchor=(1.18, 1), fontsize=fontsize)
# add an x grid
ax.xaxis.grid(True);

In [ ]:
ax = df_resources["Exports"].drop(["TOTAL.COEXBUS.A","TOTAL.CLEXBUS.A"], axis=1).plot(figsize=(14,5.5), color=plots_colors2, fontsize=fontsize, zorder=3)
ax.set_title("Export data information about four resources", fontsize=fontsize)
ax.set_xlabel("Year", fontsize=fontsize)
ax.set_ylabel("Trillion Btu", fontsize=fontsize)
# adjust the axis limits
ax.set_xlim(df_resources["Exports"].index.min(), df_resources["Exports"].index.max())
# move the legend outside the plot
ax.legend(bbox_to_anchor=(1.18, 1), fontsize=fontsize)
# add an x grid
ax.xaxis.grid(True);

In [ ]:
ax = scatter_matrix(df_resources["Imports"], figsize=(15,10), alpha=0.3)

In [ ]:
ax = scatter_matrix(df_resources["Exports"], figsize=(15,10), alpha=0.3)

Obviously, there is no connection between diferent resources.

## Conclusions

In this assignment we looked at the analysis of different resources imports and exports. As we might expect, COEXBUS had most increase during 70 years and BMEXBUS almost didn't changed at all for export theme, COIMXBUS had most increase during 70 years and BOIMBUS almost didn't changed at all for import theme.

One particular challenge of using the **US Energy Information Administration API** is that ou have to reverse the dataframe(making the year to be index instead of resource name) and choose a proper method to deal with missing data, which meant that the data collection process had to be carefully handled to build up an initial raw dataset. A considerable level of data preprocessing was also required to merge the raw data into a single dataset for each theme.

A number of interesting avenues could be considered for further analysis of this data:

The analysis could be extended to a larger number of resources to further investigation based on text descriptions.
These charts should be improved, such as adding some details on the charts or adding some pie charts, which can more intuitively reflect the characteristics of the data.
More data can also be collected to compare whether there is some correlation between different energy imports and exports.